In [47]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt

from qiskit import Aer
from tqdm.notebook import tqdm
from sklearn.datasets import load_boston
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import multiprocessing as mp

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from utils import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
def parallel(args):
    model = args[0]
    x = args[1]
    y = args[2]
    x_test = args[3]
    y_test = args[4]
    verbose = args[5]
    
    model.train(x, y, x_test=x_test, y_test=y_test, verbose = verbose)
    
    return model

# Boston Housing Data

### Load Data

In [49]:
data = load_boston()
x = data.data
y = data.target.reshape(-1, 1)
y = scaler(y, a=0, b=1)

np.random.seed(42)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=100, test_size=100)

### PCA

In [27]:
standard = StandardScaler()
x_train = standard.fit_transform(x_train)
x_test = standard.transform(x_test)

pca = PCA(n_components=4)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)

np.sum(pca.explained_variance_ratio_)

0.7885399244727873

### Normalize

In [23]:
standard = StandardScaler()
minmax = MinMaxScaler(feature_range=(-np.pi/2, np.pi/2))

x_train_dnn = standard.fit_transform(x_train)
x_test_dnn = standard.transform(x_test)

x_train_qcn = minmax.fit_transform(x_train)
x_test_qcn = minmax.transform(x_test)

In [28]:
qcn_list = loader(data_path("boston_qcn"))
y_pred = qcn_list[0].predict(x_test_qcn)

### QCN

In [7]:
np.random.seed(42)
qcn_list = []
for i in range(10):
    qcn = sequential_qnn(n_qubits = [4, 4],
                         dim = [4, 4, 1],
                         encoder= Encoder(),
                         ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                         sampler = Parity(),
                         cost = MSE(),
                         optimizer = Adam(lr=0.1),
                         shots=0)
    
    qcn_list.append([qcn, x_train_qcn, y_train, x_test_qcn, y_test, False])

qcn_list[0][5] = True    
    
with mp.Pool(10) as p:
    qcn_list = p.map(parallel, qcn_list)     
    
saver(qcn_list, data_path("boston_qcn"))

epoch: 0, loss: 0.0658711415464872
epoch: 1, loss: 0.05782016211729164
epoch: 2, loss: 0.03486954280531398
epoch: 3, loss: 0.025404502675076542
epoch: 4, loss: 0.02870207093687224
epoch: 5, loss: 0.0277165331511739
epoch: 6, loss: 0.022892339071935452
epoch: 7, loss: 0.018349265511192307
epoch: 8, loss: 0.017548911848127305
epoch: 9, loss: 0.01807855758564733
epoch: 10, loss: 0.01681401888772038
epoch: 11, loss: 0.0152901766764616
epoch: 12, loss: 0.014977027436819652
epoch: 13, loss: 0.014863512723221102
epoch: 14, loss: 0.013820230545686622
epoch: 15, loss: 0.012431797016853954
epoch: 16, loss: 0.01193566972586254
epoch: 17, loss: 0.012248559399780519
epoch: 18, loss: 0.011875344528691788
epoch: 19, loss: 0.011155846999958212
epoch: 20, loss: 0.01120868603968067
epoch: 21, loss: 0.010908207282832608
epoch: 22, loss: 0.009970868197945936
epoch: 23, loss: 0.00988450089628382
epoch: 24, loss: 0.009895840907791583
epoch: 25, loss: 0.009448122356138984
epoch: 26, loss: 0.00936559257351245

## Noisy

In [6]:
backend_santiago = pickle.load(open("backend_santiago", "rb"))

In [7]:
np.random.seed(42)
qcn_list = []
for i in range(10):
    qcn = sequential_qnn(n_qubits = [4, 4],
                         dim = [4, 4, 1],
                         encoder= Encoder(),
                         ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                         sampler = Parity(),
                         cost = MSE(),
                         optimizer = Adam(lr=0.1),
                         shots=1024,
                         backend = backend_santiago)
    
    qcn_list.append([qcn, x_train_qcn, y_train, x_test_qcn, y_test, False])

qcn_list[0][5] = True    
    
with mp.Pool(10) as p:
    qcn_list = p.map(parallel, qcn_list)     
    
saver(qcn_list, data_path("boston_qcn_noisy"))

epoch: 0, loss: 0.061074988235096876
epoch: 1, loss: 0.047489367270198875
epoch: 2, loss: 0.03161817544009362
epoch: 3, loss: 0.0227442828043243
epoch: 4, loss: 0.024619962074750732
epoch: 5, loss: 0.02357897238969214
epoch: 6, loss: 0.01944073750945668
epoch: 7, loss: 0.01852436622751495
epoch: 8, loss: 0.019840515282148197
epoch: 9, loss: 0.018524055923085155
epoch: 10, loss: 0.016698517414987825
epoch: 11, loss: 0.015245390973879969
epoch: 12, loss: 0.014591060487158507
epoch: 13, loss: 0.01468286550017934
epoch: 14, loss: 0.013591804861751605
epoch: 15, loss: 0.0132645513505347
epoch: 16, loss: 0.013092536266374
epoch: 17, loss: 0.012511456783341772
epoch: 18, loss: 0.011982249935197243
epoch: 19, loss: 0.01116775255970896
epoch: 20, loss: 0.009361032207536108
epoch: 21, loss: 0.010243736285362716
epoch: 22, loss: 0.01103238323767391
epoch: 23, loss: 0.010764223498497481
epoch: 24, loss: 0.010008054454850564
epoch: 25, loss: 0.01055735585662465
epoch: 26, loss: 0.009358021182213301

### DNN

In [6]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [4, 6, 1], activation=[Tanh(), Identity()])
    dnn.train(x_train_dnn, y_train, x_test=x_test_dnn, y_test=y_test, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("boston_dnn_pca"))
dnn_list[0].n_parameters

  0%|          | 0/10 [00:00<?, ?it/s]

37

## Hybrid

In [7]:
data = load_boston()
x = data.data
y = data.target.reshape(-1, 1)
y = scaler(y, a=0, b=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=100, test_size=100)

In [8]:
standard = StandardScaler()

x_train = standard.fit_transform(x_train)
x_test = standard.transform(x_test)

In [10]:
np.random.seed(42)
hybrid_list = []
for i in range(10):
    layer1 = Dense(n_features = 13, 
                   n_targets = 4,
                   activation = Tanh(),
                   scale=np.pi)

    layer2 = QLayer(n_qubits = 4,
                    n_features = 4, 
                    n_targets = 4, 
                    encoder = Encoder(), 
                    ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                    sampler=Parity(),
                    shots=0)

    layer3 = QLayer(n_qubits = 4,
                    n_features = 4, 
                    n_targets = 1, 
                    encoder = Encoder(), 
                    ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                    sampler=Parity(),
                    shots=0)

    layers = [layer1, layer2, layer3]
    hybrid = NeuralNetwork(layers)
    
    hybrid_list.append([hybrid, x_train, y_train, x_test, y_test, False])

hybrid_list[0][5] = True    
    
with mp.Pool(10) as p:
    hybrid_list = p.map(parallel, hybrid_list)     
    
saver(hybrid_list, data_path("boston_hybrid"))

epoch: 0, loss: 0.07490305916345207
epoch: 1, loss: 0.02624174469944178
epoch: 2, loss: 0.022680772485249922
epoch: 3, loss: 0.0225476232040407
epoch: 4, loss: 0.020191068826272897
epoch: 5, loss: 0.019810127508495576
epoch: 6, loss: 0.019715861223466212
epoch: 7, loss: 0.019066190551433056
epoch: 8, loss: 0.017297092297728633
epoch: 9, loss: 0.017524813439122378
epoch: 10, loss: 0.0177074498076755
epoch: 11, loss: 0.01748887222531164
epoch: 12, loss: 0.01635706843709467
epoch: 13, loss: 0.01546489726182827
epoch: 14, loss: 0.015463095157658096
epoch: 15, loss: 0.014333597996150662
epoch: 16, loss: 0.013164563132785688
epoch: 17, loss: 0.013034883160617157
epoch: 18, loss: 0.013150991949354884
epoch: 19, loss: 0.012857628237764407
epoch: 20, loss: 0.01242954017279705
epoch: 21, loss: 0.012525146354032162
epoch: 22, loss: 0.012527561141585916
epoch: 23, loss: 0.01204132263614635
epoch: 24, loss: 0.011410283083634792
epoch: 25, loss: 0.01118395543358558
epoch: 26, loss: 0.011016608891426

In [9]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [13, 5, 5, 1], activation = [Tanh(), Tanh(), Identity()])
    dnn.train(x_train, y_train, x_test=x_test, y_test=y_test, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("boston_dnn_full"))
dnn_list[0].n_parameters

  0%|          | 0/10 [00:00<?, ?it/s]

106

# Breast Cancer Data

## Load Data

In [42]:
data = load_breast_cancer()
x = data.data
y = data.target.reshape(-1, 1)
y = scaler(y, a=0, b=1)

np.random.seed(42)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=100, test_size=100)

### PCA

In [43]:
standard = StandardScaler()
x_train = standard.fit_transform(x_train)
x_test = standard.transform(x_test)

pca = PCA(n_components=4)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)

np.sum(pca.explained_variance_ratio_)

0.8361695759671096

### Standardize

In [20]:
standard = StandardScaler()
minmax = MinMaxScaler(feature_range=(-np.pi/2, np.pi/2))

x_train_dnn = standard.fit_transform(x_train)
x_test_dnn = standard.transform(x_test)

x_train_qcn = minmax.fit_transform(x_train)
x_test_qcn = minmax.transform(x_test)

### QCN

In [ ]:
np.random.seed(42)
qcn_list = []
for i in range(10):
    qcn = sequential_qnn(n_qubits = [4, 4],
                         dim = [4, 4, 1],
                         encoder= Encoder(),
                         ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                         sampler = Parity(),
                         cost = MSE(),
                         optimizer = Adam(lr=0.1),
                         shots=0)
    
    qcn_list.append([qcn, x_train_qcn, y_train, x_test_qcn, y_test, False])

qcn_list[0][5] = True    
    
with mp.Pool(10) as p:
    qcn_list = p.map(parallel, qcn_list)     
    
saver(qcn_list, data_path("cancer_qcn"))

## Noisy 

In [14]:
backend_santiago = pickle.load(open("backend_santiago", "rb"))

In [15]:
np.random.seed(42)
qcn_list = []
for i in range(10):
    qcn = sequential_qnn(n_qubits = [4, 4],
                         dim = [4, 4, 1],
                         encoder= Encoder(),
                         ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                         sampler = Parity(),
                         cost = MSE(),
                         optimizer = Adam(lr=0.1),
                         shots=1024,
                         backend = backend_santiago)
    
    qcn_list.append([qcn, x_train_qcn, y_train, x_test_qcn, y_test, False])

qcn_list[0][5] = True    
    
with mp.Pool(10) as p:
    qcn_list = p.map(parallel, qcn_list)     
    
saver(qcn_list, data_path("cancer_qcn_noisy"))

epoch: 0, loss: 0.2292729377746582
epoch: 1, loss: 0.19067764282226562
epoch: 2, loss: 0.17447705268859864
epoch: 3, loss: 0.15097317695617676
epoch: 4, loss: 0.13038593292236328
epoch: 5, loss: 0.10655179023742675
epoch: 6, loss: 0.09703316688537597
epoch: 7, loss: 0.08864757537841797
epoch: 8, loss: 0.08457493782043457
epoch: 9, loss: 0.07916447639465332
epoch: 10, loss: 0.07310065269470215
epoch: 11, loss: 0.07159923553466797
epoch: 12, loss: 0.06523584365844727
epoch: 13, loss: 0.06685077667236328
epoch: 14, loss: 0.06693076133728028
epoch: 15, loss: 0.07058663368225097
epoch: 16, loss: 0.06766383171081543
epoch: 17, loss: 0.06500112533569335
epoch: 18, loss: 0.06406525611877441
epoch: 19, loss: 0.06137063980102539
epoch: 20, loss: 0.06156696319580078
epoch: 21, loss: 0.06257088661193848
epoch: 22, loss: 0.06244911193847656
epoch: 23, loss: 0.06497830390930176
epoch: 24, loss: 0.05978634834289551
epoch: 25, loss: 0.061345386505126956
epoch: 26, loss: 0.05703413009643555
epoch: 27, 

### DNN

In [16]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [4, 6, 1], activation=[Tanh(), Sigmoid()])
    dnn.train(x_train_dnn, y_train, x_test=x_test_dnn, y_test=y_test, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("cancer_dnn_pca"))

  0%|          | 0/10 [00:00<?, ?it/s]

### Accuracy

#### Ideal

In [11]:
qcn_pca_list = loader(data_path("cancer_qcn"))
dnn_pca_list = loader(data_path("cancer_dnn_pca"))

acc_qcn_train = 0
acc_dnn_train = 0
acc_qcn_test = 0
acc_dnn_test = 0

for model in qcn_pca_list:
    y_pred_train = model.predict(x_train_qcn)
    acc_qcn_train += accuracy(y_pred_train, y_train)
    
    y_pred_test = model.predict(x_test_qcn)
    acc_qcn_test += accuracy(y_pred_test, y_test)

acc_qcn_train /= len(qcn_pca_list) 
acc_qcn_test /= len(qcn_pca_list) 
    
for model in dnn_pca_list:
    y_pred_train = model.predict(x_train_dnn)
    y_pred_test = model.predict(x_test_dnn)
    acc_dnn_train += accuracy(y_pred_train, y_train)
    acc_dnn_test += accuracy(y_pred_test, y_test)
    
acc_dnn_train /= len(dnn_pca_list)
acc_dnn_test /= len(dnn_pca_list)

In [12]:
print(f"QCN train acc: {acc_qcn_train:.3f}")
print(f"QCN test acc: {acc_qcn_test:.3f}")
print(f"DNN train acc: {acc_dnn_train:.3f}")
print(f"DNN test acc: {acc_dnn_test:.3f}")

QCN train acc: 0.989
QCN test acc: 0.947
DNN train acc: 1.000
DNN test acc: 0.965


#### Noisy

In [21]:
qcn_pca_list = loader(data_path("cancer_qcn_noisy"))
dnn_pca_list = loader(data_path("cancer_dnn_pca"))

acc_qcn_train = 0
acc_dnn_train = 0
acc_qcn_test = 0
acc_dnn_test = 0

for model in qcn_pca_list:
    y_pred_train = model.predict(x_train_qcn)
    acc_qcn_train += accuracy(y_pred_train, y_train)
    
    y_pred_test = model.predict(x_test_qcn)
    acc_qcn_test += accuracy(y_pred_test, y_test)

acc_qcn_train /= len(qcn_pca_list) 
acc_qcn_test /= len(qcn_pca_list) 
    
for model in dnn_pca_list:
    y_pred_train = model.predict(x_train_dnn)
    y_pred_test = model.predict(x_test_dnn)
    acc_dnn_train += accuracy(y_pred_train, y_train)
    acc_dnn_test += accuracy(y_pred_test, y_test)
    
acc_dnn_train /= len(dnn_pca_list)
acc_dnn_test /= len(dnn_pca_list)

In [22]:
print(f"QCN train acc: {acc_qcn_train:.3f}")
print(f"QCN test acc: {acc_qcn_test:.3f}")
print(f"DNN train acc: {acc_dnn_train:.3f}")
print(f"DNN test acc: {acc_dnn_test:.3f}")

QCN train acc: 0.979
QCN test acc: 0.941
DNN train acc: 1.000
DNN test acc: 0.965


## Hybrid

In [17]:
data = load_breast_cancer()
x = data.data
y = data.target.reshape(-1, 1)
y = scaler(y, a=0, b=1)

np.random.seed(42)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=100, test_size=100)

In [18]:
standard = StandardScaler()

x_train = standard.fit_transform(x_train)
x_test = standard.transform(x_test)

In [19]:
np.random.seed(42)
hybrid_list = []
for i in range(10):
    layer1 = Dense(n_features = 30, 
                   n_targets = 4,
                   activation = Tanh(),
                   scale=np.pi)

    layer2 = QLayer(n_qubits = 4,
                    n_features = 4, 
                    n_targets = 4, 
                    encoder = Encoder(), 
                    ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                    sampler=Parity(),
                    shots=0)

    layer3 = QLayer(n_qubits = 4,
                    n_features = 4, 
                    n_targets = 1, 
                    encoder = Encoder(), 
                    ansatz = Ansatz(blocks = ["entangle", "ry"], reps=2),
                    sampler=Parity(),
                    shots=0)

    layers = [layer1, layer2, layer3]
    hybrid = NeuralNetwork(layers)
    
    hybrid_list.append([hybrid, x_train, y_train, x_test, y_test, False])

hybrid_list[0][5] = True    
    
with mp.Pool(10) as p:
    hybrid_list = p.map(parallel, hybrid_list)     
    
saver(hybrid_list, data_path("cancer_hybrid"))

epoch: 0, loss: 0.22339399646443062
epoch: 1, loss: 0.23102399929656606
epoch: 2, loss: 0.20416002128632008
epoch: 3, loss: 0.20637724887955525
epoch: 4, loss: 0.20125966090467717
epoch: 5, loss: 0.19369623205279157
epoch: 6, loss: 0.1865618165679604
epoch: 7, loss: 0.17613548795399894
epoch: 8, loss: 0.16685967152924575
epoch: 9, loss: 0.15043353402611367
epoch: 10, loss: 0.13571890068823594
epoch: 11, loss: 0.1284785694424141
epoch: 12, loss: 0.11478194520588195
epoch: 13, loss: 0.10586830244152495
epoch: 14, loss: 0.11505670768832299
epoch: 15, loss: 0.10687315150767963
epoch: 16, loss: 0.09592633596754721
epoch: 17, loss: 0.09187218640634391
epoch: 18, loss: 0.07559093176805864
epoch: 19, loss: 0.07560170741343232
epoch: 20, loss: 0.07059533073729723
epoch: 21, loss: 0.06852437694655861
epoch: 22, loss: 0.06045894424953241
epoch: 23, loss: 0.05405697512665698
epoch: 24, loss: 0.04778995703803264
epoch: 25, loss: 0.04695359661731901
epoch: 26, loss: 0.04429725705462993
epoch: 27, lo

In [24]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [30, 5, 5, 1], activation=[Tanh(), Tanh(), Sigmoid()])
    dnn.train(x_train, y_train, x_test=x_test, y_test=y_test, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("cancer_dnn_full"))

  0%|          | 0/10 [00:00<?, ?it/s]

### Accuracy

In [24]:
hybrid_list = loader(data_path("cancer_hybrid"))
dnn_full_list = loader(data_path("cancer_dnn_full"))


acc_qcn_train = 0
acc_dnn_train = 0
acc_qcn_test = 0
acc_dnn_test = 0

for model in hybrid_list:
    y_pred_train = model.predict(x_train)
    acc_qcn_train += accuracy(y_pred_train, y_train)
    
    y_pred_test = model.predict(x_test)
    acc_qcn_test += accuracy(y_pred_test, y_test)

acc_qcn_train /= len(qcn_pca_list) 
acc_qcn_test /= len(qcn_pca_list) 
    
for model in dnn_full_list:
    y_pred_train = model.predict(x_train)
    acc_dnn_train += accuracy(y_pred_train, y_train)
    
    y_pred_test = model.predict(x_test)
    acc_dnn_test += accuracy(y_pred_test, y_test)
    
    
acc_dnn_train /= len(dnn_pca_list)
acc_dnn_test /= len(dnn_pca_list)

In [25]:
print(f"QCN train acc: {acc_qcn_train:.3f}")
print(f"QCN test acc: {acc_qcn_test:.3f}")
print(f"DNN train acc: {acc_dnn_train:.3f}")
print(f"DNN test acc: {acc_dnn_test:.3f}")

QCN train acc: 0.995
QCN test acc: 0.875
DNN train acc: 1.000
DNN test acc: 0.954
